In [2]:
import os
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input as preprocess_input2
from tensorflow.keras.preprocessing import image
import joblib

from sklearn.svm import LinearSVC, SVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_predict

import time
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.optimizers import Adam, RMSprop

import math
import numpy as np
import seaborn as sns
import pickle
from matplotlib import pyplot as plt
import cv2
from sklearn import metrics

In [3]:
from utils import FEATURES_FOLDER

root_dir = FEATURES_FOLDER + '/'
base_dir = root_dir + 'mel_fe/'
train_dir = base_dir + 'training/'
val_dir = base_dir + 'validation/'
test_dir = base_dir + 'test/'

In [ ]:
input_shape = (224, 224)
batch_size=32

from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(preprocessing_function=preprocess_input, validation_split=0.2)

train_generator = datagen.flow_from_directory(
        base_dir,
        target_size=(input_shape[0], input_shape[1]),
        color_mode= "rgb",
        shuffle=False,
        batch_size=batch_size,
        class_mode='sparse', 
        subset='training')

test_generator = datagen.flow_from_directory(
        base_dir,
        target_size=(input_shape[0], input_shape[1]),
        color_mode= "rgb",
        shuffle=False,
        batch_size=batch_size,
        class_mode='sparse',
        subset='validation')

y_train = train_generator.classes
y_test = test_generator.classes

In [4]:
#base_model = ResNet50(weights='imagenet')
base_model = VGG16(weights='imagenet')
base_model.summary()

# resnet50(conv5_block1_2_relu, avg_pool)
# vgg16(block5_pool, fc1, fc2)
base_model = keras.Model(base_model.input, base_model.get_layer("block5_pool").output)
base_model.trainable = False
base_model.summary()

2022-01-23 23:50:15.093282: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-23 23:50:15.112970: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-23 23:50:15.113134: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-23 23:50:15.113579: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

# Post-Quantization

In [5]:
base_model.save("model.h5")
print("Saved model to disk")

# without quantization
base_model = tf.keras.models.load_model('model.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(base_model)
tflite_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_model)

# with quantization
base_model = tf.keras.models.load_model('model.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(base_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()
open("converted_quant_model.tflite", "wb").write(tflite_quant_model)

print("Float model in Mb:", os.path.getsize('converted_model.tflite') / float(2**20))
print("Quantized model in Mb:", os.path.getsize('converted_quant_model.tflite') / float(2**20))
print("Compression ratio:", os.path.getsize('converted_model.tflite')/os.path.getsize('converted_quant_model.tflite'))

Saved model to disk


2022-01-23 23:50:24.329560: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /tmp/tmpde6w372h/assets


2022-01-23 23:50:25.271808: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:363] Ignored output_format.


2022-01-23 23:50:25.271825: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:366] Ignored drop_control_dependency.
2022-01-23 23:50:25.272419: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmpde6w372h
2022-01-23 23:50:25.274006: I tensorflow/cc/saved_model/reader.cc:107] Reading meta graph with tags { serve }
2022-01-23 23:50:25.274019: I tensorflow/cc/saved_model/reader.cc:148] Reading SavedModel debug info (if present) from: /tmp/tmpde6w372h
2022-01-23 23:50:25.280869: I tensorflow/cc/saved_model/loader.cc:210] Restoring SavedModel bundle.
2022-01-23 23:50:25.352930: I tensorflow/cc/saved_model/loader.cc:194] Running initialization op on SavedModel bundle at path: /tmp/tmpde6w372h
2022-01-23 23:50:25.364972: I tensorflow/cc/saved_model/loader.cc:283] SavedModel load for tags { serve }; Status: success: OK. Took 92654 microseconds.
2022-01-23 23:50:25.385935: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:237] disabling MLIR 

INFO:tensorflow:Assets written to: /tmp/tmpy_rc6n0g/assets


INFO:tensorflow:Assets written to: /tmp/tmpy_rc6n0g/assets
2022-01-23 23:50:26.939343: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:363] Ignored output_format.


Float model in Mb: 56.138916015625
Quantized model in Mb: 14.133407592773438
Compression ratio: 3.9720722442285914


2022-01-23 23:50:26.939360: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:366] Ignored drop_control_dependency.
2022-01-23 23:50:26.939486: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmpy_rc6n0g
2022-01-23 23:50:26.941034: I tensorflow/cc/saved_model/reader.cc:107] Reading meta graph with tags { serve }
2022-01-23 23:50:26.941045: I tensorflow/cc/saved_model/reader.cc:148] Reading SavedModel debug info (if present) from: /tmp/tmpy_rc6n0g
2022-01-23 23:50:26.945799: I tensorflow/cc/saved_model/loader.cc:210] Restoring SavedModel bundle.
2022-01-23 23:50:26.987757: I tensorflow/cc/saved_model/loader.cc:194] Running initialization op on SavedModel bundle at path: /tmp/tmpy_rc6n0g
2022-01-23 23:50:27.000923: I tensorflow/cc/saved_model/loader.cc:283] SavedModel load for tags { serve }; Status: success: OK. Took 61438 microseconds.


In [ ]:
def extract_features(model, datagen):
  features = model.predict_generator(datagen)
  features = features.reshape((features.shape[0], -1))
  return features

def extract_features_tflite(model):
  # Classify image with TensorFlow Lite.
  f = tf.contrib.lite.Interpreter(f'{model.name}.tflite')
  f.allocate_tensors()
  i = f.get_input_details()[0]
  o = f.get_output_details()[0]
  f.set_tensor(i['index'], x)
  f.invoke()
  features = f.get_tensor(o['index'])
  features = features.reshape((features.shape[0], -1))
  return 
def pre_process(features, std_scaler, pca_scaler):
  if std_scaler == None:
    std_scaler = StandardScaler()
    std_scaler.fit(features)

  if pca_scaler == None:
    pca_scaler = PCA(0.90)
    pca_scaler.fit(features)

  features = std_scaler.transform(features)
  features = pca_scaler.transform(features)
  return features, std_scaler, pca_scaler

def test(clf, features, labels):
  pred = clf.predict(features)

  fig, ax = plt.subplots(figsize=(6, 6))
  disp = metrics.ConfusionMatrixDisplay(metrics.confusion_matrix(labels, pred), display_labels=range(8))
  disp.plot(ax=ax, colorbar=False,cmap='Blues')

  print('{}'.format(metrics.classification_report(labels, pred)))
  print('Accuracy: {}'.format(metrics.accuracy_score(labels, pred)))


In [6]:
#[std_scaler, pca_scaler, clf] = joblib.load('best_resnet50.sav')
[std_scaler, pca_scaler, clf] = joblib.load('best_vgg16.sav')
X_test, _, _ = pre_process(extract_features(base_model, test_generator), std_scaler, pca_scaler)
print(X_test.shape)
test(clf, X_test, y_test)

NameError: name 'pre_process' is not defined